# Laboratorio 3


## Integrantes
- Gustavo Cruz 22779
- Mathew Cordero 22982
- Pedro Guzmán 22111

## Repositorio

[Link al Repositorio](https://github.com/donmatthiuz/ModelacionSimu/tree/lab3)




## Ejercicio 1  

Implementar los siguientes métodos de **descenso gradiente** (naïve = tamaño de paso α constante):  

1. Descenso gradiente naïve con dirección de descenso **aleatoria**.  
2. Descenso gradiente **máximo** naïve.  
3. Descenso gradiente de **Newton**, con Hessiano exacto.  
4. Un método de **gradiente conjugado** (Fletcher-Reeves, Hestenes-Stiefel o Polak-Ribiere).  
5. El método **BFGS**.  

### Requerimientos

En cada uno de los métodos, la función debe recibir los siguientes argumentos:

- La función objetivo `f`.  
- El gradiente de la función objetivo `df`.  
- El Hessiano `ddf` (cuando sea necesario).  
- Un punto inicial `x0 ∈ R^n`.  
- El tamaño de paso `α > 0`.  
- El número máximo de iteraciones `maxIter`.  
- La tolerancia `ε`.  
- Un criterio de paro.  

### Resultados esperados

Los algoritmos deben devolver:  

- La mejor solución encontrada `best` (la última de las aproximaciones calculadas).  
- La secuencia de iteraciones `xk`.  
- La secuencia de valores `f(xk)`.  
- La secuencia de errores en cada paso (según el error de su criterio de paro).  

Además, es deseable indicar:  

- El número de iteraciones efectuadas por el algoritmo.  
- Si se obtuvo o no la **convergencia** del método.  


Lo primero que vamos a hacer es definir una clase con funciones que representan cada metodo|

In [7]:
import numpy as np
import matplotlib.pyplot as plt

class DescensoGradiente:
    def __init__(self, F, df):
        self.F = F     
        self.df = df   

    # Función para graficar resultados
    def plot_results(self, fk, errors, metodo):
        plt.figure(figsize=(12,5))
        
        plt.subplot(1,2,1)
        plt.plot(fk, marker='o')
        plt.title(f"{metodo}: Evolución de F(x)")
        plt.xlabel("Iteración")
        plt.ylabel("F(x)")
        plt.grid(True)

        plt.subplot(1,2,2)
        plt.plot(errors, marker='o', color='r')
        plt.title(f"{metodo}: Evolución del Error")
        plt.xlabel("Iteración")
        plt.ylabel("Error")
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()

    # Método de descenso aleatorio
    def naive_random(self, x0 = np.zeros(2), alpha = 0.01, maxIter = 10000, tol = 1e-10):
        xk = [x0]             
        fk = [self.F(x0)]
        errors = [np.linalg.norm(self.df(x0)) if self.df else np.inf]

        x = np.array(x0, dtype=float)
        for k in range(maxIter):
            # Dirección aleatoria normalizada
            d = np.random.randn(*x.shape)
            d = d / np.linalg.norm(d)

            x_new = x - alpha * d
            f_new = self.F(x_new)

            xk.append(x_new)
            fk.append(f_new)
            error = np.abs(fk[-1] - fk[-2])
            errors.append(error)

            if error < tol:
                print(f"Descenso Aleatorio: Convergió después de {k+1} iteraciones con error {error}")
                self.plot_results(fk, errors, "Descenso Aleatorio")
                return x_new, xk, fk, errors, k+1, True

            x = x_new

        print("Descenso Aleatorio: No convergió")
        self.plot_results(fk, errors, "Descenso Aleatorio")
        return x, xk, fk, errors, maxIter, False
    
    # Método de gradiente más pronunciado
    def steepest_descent(self, x0 = np.zeros(2), alpha = 0.01, maxIter = 10000, tol = 1e-10): 
        xk = [x0]             
        fk = [self.F(x0)]
        errors = [np.linalg.norm(self.df(x0))]

        x = np.array(x0, dtype=float)
        for k in range(maxIter):
            grad = self.df(x)
            if np.linalg.norm(grad) < tol:
                print(f"Descenso por Gradiente: Convergió (gradiente < tol) después de {k} iteraciones")
                self.plot_results(fk, errors, "Descenso por Gradiente")
                return x, xk, fk, errors, k, True

            # Actualización del punto
            x_new = x - alpha * grad
            f_new = self.F(x_new)

            xk.append(x_new)
            fk.append(f_new)
            error = np.abs(fk[-1] - fk[-2])
            errors.append(error)

            if error < tol:
                print(f"Descenso por Gradiente: Convergió (ΔF < tol) después de {k+1} iteraciones")
                self.plot_results(fk, errors, "Descenso por Gradiente")
                return x_new, xk, fk, errors, k+1, True

            x = x_new

        print("Descenso por Gradiente: No convergió")
        self.plot_results(fk, errors, "Descenso por Gradiente")
        return x, xk, fk, errors, maxIter, False

    # Método de Newton
    def newton_method(self, ddf, x0=np.zeros(2), maxIter=10000, tol=1e-10):
        xk = [x0]
        fk = [self.F(x0)]
        errors = [np.linalg.norm(self.df(x0))]
        x = np.array(x0, dtype=float)

        for iteration in range(maxIter):
            grad = self.df(x)
            H = ddf(x)

            if np.linalg.norm(grad) < tol:
                print(f"Método de Newton: Convergió (gradiente < tol) después de {iteration} iteraciones")
                self.plot_results(fk, errors, "Método de Newton")
                return x, xk, fk, errors, iteration, True

            try:
                delta_x = np.linalg.solve(H, -grad)
            except np.linalg.LinAlgError:
                print(f"Método de Newton: Hessiano singular en iteración {iteration}")
                self.plot_results(fk, errors, "Método de Newton")
                return x, xk, fk, errors, iteration, False

            x_new = x + delta_x
            f_new = self.F(x_new)

            xk.append(x_new)
            fk.append(f_new)
            errors.append(np.linalg.norm(delta_x))

            if np.linalg.norm(delta_x) < tol:
                print(f"Método de Newton: Convergió (Δx < tol) después de {iteration+1} iteraciones")
                self.plot_results(fk, errors, "Método de Newton")
                return x_new, xk, fk, errors, iteration+1, True

            x = x_new

        print("Método de Newton: No convergió")
        self.plot_results(fk, errors, "Método de Newton")
        return x, xk, fk, errors, maxIter, False

    # Gradiente conjugado Fletcher-Reeves
    def conjugate_gradient_flerev(self, x0=np.zeros(2), alpha=0.01, maxIter=1000, tol=1e-10):
        xk = [np.array(x0, dtype=float)]
        fk = [self.F(x0)]
        errors = [np.linalg.norm(self.df(x0))]

        x = np.array(x0, dtype=float)
        grad = self.df(x)
        d = -grad

        for k in range(maxIter):
            grad_old = grad.copy()

            x_new = x + alpha * d
            grad = self.df(x_new)

            xk.append(x_new)
            fk.append(self.F(x_new))
            errors.append(np.linalg.norm(grad))

            if np.linalg.norm(grad) < tol:
                print(f"Gradiente Conjugado (Fletcher-Reeves): Convergió después de {k+1} iteraciones")
                self.plot_results(fk, errors, "Gradiente Conjugado (F-R)")
                return x_new, xk, fk, errors, k+1, True

            beta = np.dot(grad, grad) / np.dot(grad_old, grad_old)
            d = -grad + beta * d
            x = x_new

        print("Gradiente Conjugado (Fletcher-Reeves): No convergió")
        self.plot_results(fk, errors, "Gradiente Conjugado (F-R)")
        return x, xk, fk, errors, maxIter, False

    # Método BFGS
    def bfgs_method(self, x0=np.zeros(2), tol=1e-6, maxIter=1000):
        x = np.array(x0, dtype=float)
        n = len(x)
        H = np.eye(n)
        xk = [x.copy()]
        fk = [self.F(x)]
        errors = [np.linalg.norm(self.df(x))]

        for k in range(maxIter):
            grad = self.df(x)
            d = - H.dot(grad)
            x_new = x + d
            grad_new = self.df(x_new)

            xk.append(x_new.copy())
            fk.append(self.F(x_new))
            errors.append(np.linalg.norm(grad_new))

            if np.linalg.norm(x_new - x) < tol:
                print(f"BFGS: Convergió (Δx < tol) después de {k+1} iteraciones")
                self.plot_results(fk, errors, "BFGS")
                return x_new, xk, fk, errors, k+1, True

            s = x_new - x
            y = grad_new - grad
            rho = 1.0 / np.dot(y, s)
            Hy = H.dot(y)
            H += np.outer(s, s) * rho - np.outer(Hy, Hy) / np.dot(y, Hy)

            x = x_new

        print("BFGS: No convergió")
        self.plot_results(fk, errors, "BFGS")
        return x, xk, fk, errors, maxIter, False
